# TransForest: Quick Start Guide

## What is TransForest?

TransForest 🌱 is a Python library written in Rust that improves LLM response quality using **ensemble methods**. Instead of getting one response from your LLM, it:
1. Generates multiple responses
2. Analyzes them using different algorithms
3. Selects the best one automatically

Think of it as having multiple experts give their opinion, then choosing the best answer.

## Installation and Setup

### Installation
```bash
pip install transforest groq
```

### API Key Setup
Get your Groq API key from [https://console.groq.com/keys](https://console.groq.com/keys) and set it as an environment variable:

```bash
export GROQ_API_KEY="your-api-key-here"
```

Or set it in your notebook:

In [11]:
import os
import pprint
import transforest
from groq import Groq
import pprint

# Set your Groq API key here (or use environment variable)
# os.environ["GROQ_API_KEY"] = "your-api-key-here"

# Initialize Groq client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

### First Example with Groq
Here's the simplest way to use Transforest with Groq:

In [21]:
@transforest.minimum_bayes_risk  # This decorator improves response quality
def groq_llm(prompt, model="llama-3.1-8b-instant", temperature=1):
    """LLM function using Groq API"""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=1024
    )
    return response.choices[0].message.content

# Use it like any normal function
result = groq_llm("What have been the capitals since the founding of Italy? Stick to 20 words.")

pprint.pprint(result)

{'all_responses': [{'avg_distance': 0.7113594653772658,
                    'execution_time': 0.299085745,
                    'index': 0,
                    'response': 'Florence (1865-1870), then Rome '
                                '(1870-present), were key capital cities since '
                                'the unification of Italy in 1861.'},
                   {'avg_distance': 0.6547242595304493,
                    'execution_time': 0.210119627,
                    'index': 1,
                    'response': 'Turin (1861-1865), Florence (1865-1871), Rome '
                                '(1871-present), with brief periods in Naples '
                                '(1861) and Gaeta (1861).'},
                   {'avg_distance': 0.6883243448388412,
                    'execution_time': 0.315194744,
                    'index': 2,
                    'response': 'Florence (1865-1871), Rome (1871-present) are '
                                'the main capitals since unifi

**That's it!** Just add `@transforest.minimum_bayes_risk` above your Groq function and it automatically improves response quality, stability, and consistency.

### What happened?
1. The decorator called your function 5 times (default)
2. It analyzed all responses using cosine similarity
3. It selected the most "representative" response
4. It returned detailed results including the best response

## Decorators and Ensemble Methods

Transforest provides three different ensemble methods. Here's when to use each:

### 1. Minimum Bayes Risk (MBR)

**What it does:** Selects the response with the lowest average distance to all other responses.

**When to use:**
- Non punctual answers
- Complex questions requiring consistency answers
- When you want the most "representative" response

**Best for:** Quality over speed

In [26]:
@transforest.minimum_bayes_risk(num_calls=3)
def technical_groq(prompt, model="llama-3.1-8b-instant"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an expert C++ language software developer. Always respond only with code to solve the questions submitted to you."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=1024
    )
    return response.choices[0].message.content

result = technical_groq("Implement a function to exchange values between two variables. Try to be as concise as possible.")
pprint.pprint(result)

{'all_responses': [{'avg_distance': 0.036674172654825976,
                    'execution_time': 0.467627777,
                    'index': 0,
                    'response': '```cpp\n'
                                '#include <iostream>\n'
                                '\n'
                                'void exchange(int &a, int &b) {\n'
                                '    a = a + b;\n'
                                '    b = a - b;\n'
                                '    a = a - b;\n'
                                '}\n'
                                '\n'
                                'int main() {\n'
                                '    int x = 5;\n'
                                '    int y = 10;\n'
                                '\n'
                                '    std::cout << "Before exchange: x = " << x '
                                '<< ", y = " << y << std::endl;\n'
                                '    exchange(x, y);\n'
                                ' 

### 2. Majority Voting

**What it does:** Selects the most frequently occurring response.

**When to use:**
- Questions with clear answers (e.g., classification, name entities recognition, etc.)
- When consistency is more important than creativity
- Reducing hallucinations

**Best for:** Speed and consistency

In [ ]:
@transforest.majority_voting(num_calls=3)
def factual_groq(prompt, model="llama-3.1-8b-instant"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a classifier of user comments on social networks. Always respond only with ‘positive’ or ‘negative’. Label the comment based on the status of the comment."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1,
        max_tokens=512
    )
    return response.choices[0].message.content

result = factual_groq("Well… that’s certainly unforgettable.")
pprint.pprint(result)

{'all_responses': [{'execution_time': 0.216489559,
                    'index': 0,
                    'response': 'Negative',
                    'vote_count': 5},
                   {'execution_time': 0.212581566,
                    'index': 1,
                    'response': 'Negative',
                    'vote_count': 5},
                   {'execution_time': 0.157611187,
                    'index': 2,
                    'response': 'Negative',
                    'vote_count': 5},
                   {'execution_time': 0.150980145,
                    'index': 3,
                    'response': 'Negative',
                    'vote_count': 5},
                   {'execution_time': 0.184721896,
                    'index': 4,
                    'response': 'Negative',
                    'vote_count': 5}],
 'selected_index': 0,
 'selected_response': 'Negative',
 'total_execution_time': 0.922400098,
 'vote_counts': {'Negative': 5}}


### 3. Blender

**What it does:** Uses PairRanker for ranking + GenFuser for combining responses.

**When to use:**
- Research or analysis tasks requiring multiple perspectives
- When you want response fusion, not just selection
- When consistency and answer integrity are not the priority

**Best for:** Quality and multiple perspectives but slower

In [16]:
inference_config = {
    "provider":"https://api.groq.com/openai/v1/chat/completions",
    "api_key":os.getenv("GROQ_API_KEY"),
    "model":"llama-3.1-8b-instant",
}

@transforest.blender(num_calls=7, top_k=3, inference_config=inference_config)
def research_groq(prompt, model="llama-3.1-8b-instant"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a travel agent. You always provide only a rough itinerary that reflects the request. Stick to 20 words."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.8,
        max_tokens=1024
    )
    return response.choices[0].message.content

result = research_groq("Give me an itinerary for a young, sporty couple who want to spend two days in Liguria.")
pprint.pprint(result)

{'all_responses': [{'execution_time': 0.362669749,
                    'index': 0,
                    'ranking_score': 0.047619047619047616,
                    'response': "Explore Genoa's historic center, hike "
                                "Portofino's scenic trails, and enjoy the "
                                'local seafood in Santa Margherita Ligure.'},
                   {'execution_time': 0.284667668,
                    'index': 1,
                    'ranking_score': 0.19047619047619047,
                    'response': 'Hike Cinque Terre trails, swim in the '
                                'Mediterranean, dine in Monterosso, and relax '
                                'in Santa Margherita Ligure for two days.'},
                   {'execution_time': 0.383517761,
                    'index': 2,
                    'ranking_score': 0.14285714285714285,
                    'response': 'Hike Cinque Terre trails, visit Portofino, '
                                "explore 

## Complex Use Cases

Now let's see real-world scenarios where Transforest decorators shine:

### Use Case 1: Temperature Variation Strategy

**What it does:** Different temperatures for different types of responses

**When to use:**
- .

**Best for:** .

In [ ]:
import random

@transforest.minimum_bayes_risk(num_calls=3)
def technical_groq():
    temperature = random.uniform(0.1, 1)
    print(f"Temperature: {temperature}")
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "user", "content": "You are Homer, the Greek poet. Write four lines about one of Ulysses' adventures."}
        ],
        temperature=temperature,
        max_tokens=1024
    )
    return response.choices[0].message.content

result = technical_groq()
pprint.pprint(result)

Temperature: 0.5760885128515892
Temperature: 0.7930758942458829
Temperature: 0.21408236349916346
{'all_responses': [{'avg_distance': 0.7425989984497254,
                    'execution_time': 0.36623958,
                    'index': 0,
                    'response': "Mortal, I shall sing of Ulysses' plight,\n"
                                "As he escaped the Sirens' deadly night.\n"
                                "Their song, a siren's call, a temptation "
                                'sweet,\n'
                                'Lured sailors to their doom, where death did '
                                'meet.'},
                   {'avg_distance': 0.8062185990914266,
                    'execution_time': 0.324801478,
                    'index': 1,
                    'response': '"Mortal Ulysses, oh so brave and true,\n'
                                'In the land of the Lotus-Eaters, his spirit '
                                'anew,\n'
                                'He 

### Use Case 2: Multi-Model Ensemble

Using different models and selecting the best response:

In [18]:
import random

@transforest.minimum_bayes_risk(num_calls=3)
def technical_groq():
    models = ["llama-3.1-8b-instant", "llama-3.3-70b-versatile", "openai/gpt-oss-20b", "openai/gpt-oss-120b"]
    model = random.choice(models)
    print(f"Model: {model}")
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": "You are Homer, the Greek poet. Write four lines about one of Ulysses' adventures."}
        ],
        temperature=0.1,
        max_tokens=1024
    )
    return response.choices[0].message.content

result = technical_groq()
pprint.pprint(result)

Model: llama-3.1-8b-instant
Model: llama-3.1-8b-instant
Model: openai/gpt-oss-20b
{'all_responses': [{'avg_distance': 0.4288104039970955,
                    'execution_time': 0.295586317,
                    'index': 0,
                    'response': "Mortal, I sing of Ulysses' might,\n"
                                "As he sailed through the whirlpool's deadly "
                                'night,\n'
                                "Charybdis' jaws did fiercely roar and snap,\n"
                                'But Ulysses steered clear, his wits '
                                'unclapped.'},
                   {'avg_distance': 0.41478470959813546,
                    'execution_time': 0.286527567,
                    'index': 1,
                    'response': "Mortal, I sing of Ulysses' might,\n"
                                "As he sailed through the whirlpool's deadly "
                                'night,\n'
                                "Charybdis' jaws did fier